# "Predicting Car Prices using the K Nearest Neighbors Algorithm"

> "I apply the K Nearest Neighbors (KNN) regression algorithm to predict the prices of cars based on various features, including (expound here)"

- author: Migs Germar
- toc: true
- branch: master
- badges: true
- comments: true
- categories: [python, pandas, numpy, sklearn]
- hide: true
- search_exclude: true
- image: images/logo.png

<center><img src = "https://miguelahg.github.io/mahg-data-science/images/logo.png" alt = ""></center>

<center><a href = "https://unsplash.com/photos/gKXKBY-C-Dk">Unsplash | Photographer Name</a></center>

# Introduction

Paragraphs giving an overview of the project.

> Note: I wrote this notebook by following a guided project on the [Dataquest](https://www.dataquest.io/) platform, specifically the [Guided Project: Predicting Car Prices](https://app.dataquest.io/c/36/m/155/guided-project%3A-predicting-car-prices/1/introduction-to-the-data-set) The general project flow and research questions were guided by Dataquest.

# Packages

Below are the packages used in this project.

In [89]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, cross_val_score
import re

# Data Inspection and Cleaning

The dataset for this project is the Automobile Data Set by Schlimmer (1987), from the UCI Machine Learning Repository. It contains data on 26 features of hundreds of cars. A summary of the features and their data types is shown below.

In [90]:
#collapse-hide
# Data dictionary from documentation.
data_dict = """1. symboling: -3, -2, -1, 0, 1, 2, 3.
2. normalized-losses: continuous from 65 to 256.
3. make:
alfa-romero, audi, bmw, chevrolet, dodge, honda,
isuzu, jaguar, mazda, mercedes-benz, mercury,
mitsubishi, nissan, peugot, plymouth, porsche,
renault, saab, subaru, toyota, volkswagen, volvo
4. fuel-type: diesel, gas.
5. aspiration: std, turbo.
6. num-of-doors: four, two.
7. body-style: hardtop, wagon, sedan, hatchback, convertible.
8. drive-wheels: 4wd, fwd, rwd.
9. engine-location: front, rear.
10. wheel-base: continuous from 86.6 120.9.
11. length: continuous from 141.1 to 208.1.
12. width: continuous from 60.3 to 72.3.
13. height: continuous from 47.8 to 59.8.
14. curb-weight: continuous from 1488 to 4066.
15. engine-type: dohc, dohcv, l, ohc, ohcf, ohcv, rotor.
16. num-of-cylinders: eight, five, four, six, three, twelve, two.
17. engine-size: continuous from 61 to 326.
18. fuel-system: 1bbl, 2bbl, 4bbl, idi, mfi, mpfi, spdi, spfi.
19. bore: continuous from 2.54 to 3.94.
20. stroke: continuous from 2.07 to 4.17.
21. compression-ratio: continuous from 7 to 23.
22. horsepower: continuous from 48 to 288.
23. peak-rpm: continuous from 4150 to 6600.
24. city-mpg: continuous from 13 to 49.
25. highway-mpg: continuous from 16 to 54.
26. price: continuous from 5118 to 45400."""

# Use regex to extract column names from data dictionary.
col_names = re.findall(
    pattern = r"^[0-9]{1,2}\. ([a-z\-]+):",
    string = data_dict,
    # Use multiline flag so that ^ indicates the start of a line.
    flags = re.MULTILINE,
)

# Read data file and add column names.
cars_df = pd.read_csv(
    "./private/Car-Prices-KNN-Files/imports-85.data",
    names = col_names,
)

cars_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized-losses  205 non-null    object 
 2   make               205 non-null    object 
 3   fuel-type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num-of-doors       205 non-null    object 
 6   body-style         205 non-null    object 
 7   drive-wheels       205 non-null    object 
 8   engine-location    205 non-null    object 
 9   wheel-base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb-weight        205 non-null    int64  
 14  engine-type        205 non-null    object 
 15  num-of-cylinders   205 non-null    object 
 16  engine-size        205 non

There are 205 cars and 26 features. Most of the features directly describe physical characteristics of the cars. Some exceptions are "symboling" and "normalized-losses", which are values related to car insurance and are beyond the scope of this project. Also, the "price" column provides the price of each car in USD.

Let us look at the first five rows.

In [91]:
cars_df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


If we compare the data type of each column to its contents, several opportunities for data cleaning can be seen. For example, the "normalized-losses" feature is listed as an object-type column because it contains both strings and numbers. However, the strings in the column are question marks (?). Rather than being categories, these may be placeholders for missing data. This problem applies to several other columns, not just this one.

Furthermore, in some columns like "num-of-doors", numbers are written as words. For example, 2 is written as "two". Since the numbers are in string format, these cannot be used in the K Nearest Neighbors model.

Thus, in summary, the following cleaning steps have to be performed:

- Replace question mark strings ("?") with null values (NaN). These are the proper way to indicate missing values.
- Convert several object columns, like "normalized-losses", into numeric columns.
- Replace numbers written as words with their proper numeric equivalents. For example, replace "four" with 4.

These were performed in the following code cell.

In [92]:
#collapse-hide
# Clean the data.

# Replace ? with NaN since these are placeholders.
cars_df = cars_df.replace("?", np.nan)

# Change this object column to float type.
obj_to_numeric = [
    "normalized-losses",
    "bore",
    "stroke",
    "horsepower",
    "peak-rpm",
    "price",
]

for col in obj_to_numeric:
    cars_df[col] = pd.to_numeric(cars_df[col], errors = "coerce")

# Replace strings with numerical equivalents.
cars_df["num-of-doors"] = cars_df["num-of-doors"].replace(
    {
        "four": 4.0,
        "two": 2.0,
    }
)
cars_df["num-of-cylinders"] = cars_df["num-of-cylinders"].replace(
    {
        "four": 4,
        "six": 6,
        "five": 5,
        "eight": 8,
        "two": 2,
        "three": 3,
        "twelve": 12,
    }
)

cars_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized-losses  164 non-null    float64
 2   make               205 non-null    object 
 3   fuel-type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num-of-doors       203 non-null    float64
 6   body-style         205 non-null    object 
 7   drive-wheels       205 non-null    object 
 8   engine-location    205 non-null    object 
 9   wheel-base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb-weight        205 non-null    int64  
 14  engine-type        205 non-null    object 
 15  num-of-cylinders   205 non-null    int64  
 16  engine-size        205 non

The new summary of columns is shown above. Several columns which were once "object" columns are now numeric. Also, since we replaced "?" placeholders with null values, we can now see that some columns have missing values.

In [93]:
#collapse-hide
null_percs = (
    cars_df
    .isnull()
    .sum()
    .divide(cars_df.shape[0])
    .multiply(100)
)

null_percs.loc[null_percs > 0]

normalized-losses    20.00000
num-of-doors          0.97561
bore                  1.95122
stroke                1.95122
horsepower            0.97561
peak-rpm              0.97561
price                 1.95122
dtype: float64

The table above shows the percentage of missing values in each column that has them. In particular, "normalized-losses" has missing values in 20% of the observations. Thus, we will have to drop this column from the dataset. This is better than the alternative, which is to delete all rows where "normalized-losses" is missing.

As for the other 6 columns, we will use listwise deletion. This means that we will drop all rows with missing values in any of those columns.

In [94]:
#collapse-hide
cars_df = (
    cars_df
    .drop("normalized-losses", axis = 1)
    .dropna(
        subset = [
            "num-of-doors",
            "bore",
            "stroke",
            "horsepower",
            "peak-rpm",
            "price",
        ]
    )
)

num_null = cars_df.isnull().sum().sum()
print(f"Total number of missing values: {num_null}")
print(f"New shape of dataset: {cars_df.shape}")

Total number of missing values: 0
New shape of dataset: (193, 25)


Now, there are no more missing values in the dataset. There are 193 rows and 25 columns left.

# The K Nearest Neighbors Algorithm

